# Phase 3: 세그멘테이션

이 노트북에서 배울 내용:
- RANSAC을 이용한 평면 추출
- DBSCAN 클러스터링
- 씬 세그멘테이션 파이프라인

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import open3d as o3d

from src.data import PointCloudLoader
from src.preprocessing import Preprocessor
from src.segmentation import Segmenter
from src.utils import Visualizer

## 1. 데이터 준비

In [ ]:
loader = PointCloudLoader()
preprocessor = Preprocessor()
segmenter = Segmenter()

# 샘플 씬 생성 및 전처리
scene = loader.create_sample_scene(add_noise=True)
scene = preprocessor.full_pipeline(scene, voxel_size=0.03)

Visualizer.show(scene, "Preprocessed Scene")

## 2. RANSAC 평면 추출

**RANSAC (Random Sample Consensus)**

1. 무작위로 3개 점 선택
2. 이 점들로 평면 모델 피팅
3. 모델에 가까운 점(inliers) 수 계산
4. 반복하여 가장 많은 inliers를 가진 모델 선택

**장점**: 이상치(outlier)에 강건함

In [ ]:
# 단일 평면 추출
plane, remaining = segmenter.extract_plane(
    scene,
    distance_threshold=0.02,  # 평면까지 최대 거리
    num_iterations=1000       # RANSAC 반복 횟수
)

In [ ]:
# 평면 방정식 해석
a, b, c, d = plane.equation
print(f"평면 방정식: {a:.3f}x + {b:.3f}y + {c:.3f}z + {d:.3f} = 0")
print(f"법선 벡터: ({a:.3f}, {b:.3f}, {c:.3f})")

# 법선이 Z축에 가까우면 바닥/천장
if abs(c) > 0.9:
    print("→ 수평면 (바닥 또는 천장)")
elif abs(a) > 0.9 or abs(b) > 0.9:
    print("→ 수직면 (벽)")

In [ ]:
# 시각화: 평면(회색) + 나머지(원래 색)
Visualizer.show_multiple([plane.points, remaining], "Plane (gray) + Objects")

## 3. 여러 평면 추출

In [ ]:
# 순차적으로 여러 평면 추출
planes, objects = segmenter.extract_multiple_planes(
    scene,
    max_planes=2,    # 최대 평면 수
    min_points=100   # 평면당 최소 점 수
)

In [ ]:
# 각 평면에 다른 색상 적용
colors = [[0.8, 0.8, 0.8], [0.6, 0.6, 0.6]]
geometries = []

for i, plane in enumerate(planes):
    plane.points.paint_uniform_color(colors[i % len(colors)])
    geometries.append(plane.points)

geometries.append(objects)

Visualizer.show_multiple(geometries, "Multiple Planes + Objects")

## 4. DBSCAN 클러스터링

**DBSCAN (Density-Based Spatial Clustering)**

밀도 기반으로 점들을 그룹화합니다.

**파라미터**:
- `eps`: 이웃 판정 거리 (반경)
- `min_points`: 클러스터 최소 점 수

**특징**: 클러스터 개수를 미리 지정할 필요 없음

In [ ]:
# 평면 제거 후 남은 점들 클러스터링
clusters = segmenter.cluster_dbscan(
    objects,
    eps=0.1,          # 이웃 거리
    min_points=10     # 최소 클러스터 크기
)

In [ ]:
# 클러스터 정보
for cluster in clusters:
    bbox_extent = cluster.bbox.get_extent()
    print(f"클러스터 {cluster.label}:")
    print(f"  점 수: {len(cluster.indices):,}")
    print(f"  중심: ({cluster.centroid[0]:.2f}, {cluster.centroid[1]:.2f}, {cluster.centroid[2]:.2f})")
    print(f"  크기: {bbox_extent[0]:.2f} x {bbox_extent[1]:.2f} x {bbox_extent[2]:.2f}")
    print()

In [ ]:
# 클러스터 시각화 (각각 다른 색상 + 바운딩 박스)
geometries = []
for cluster in clusters:
    geometries.append(cluster.points)
    geometries.append(cluster.bbox)

Visualizer.show_multiple(geometries, "Clusters with Bounding Boxes")

## 5. eps 파라미터의 영향

In [ ]:
# eps 값에 따른 클러스터 수 변화
for eps in [0.05, 0.1, 0.2, 0.3]:
    clusters = segmenter.cluster_dbscan(objects, eps=eps, min_points=10)
    print(f"eps={eps}: {len(clusters)}개 클러스터")

## 6. 전체 씬 세그멘테이션 파이프라인

In [ ]:
# 원본 씬 다시 로드
scene = loader.create_sample_scene(add_noise=True)
scene = preprocessor.full_pipeline(scene, voxel_size=0.03)

# 전체 세그멘테이션
planes, clusters = segmenter.segment_scene(scene)

In [ ]:
# 최종 결과 시각화
segmenter.visualize_segmentation(planes, clusters, show_bbox=True)

## 연습 문제

1. `distance_threshold`를 변경하며 평면 추출 결과를 비교해보세요.
2. 객체가 서로 가까이 있는 씬을 만들고, `eps`를 조절해 분리해보세요.
3. 추출된 클러스터 중 가장 큰 것의 바운딩 박스 크기를 출력해보세요.

In [ ]:
# 연습 코드 작성
